<a href="https://colab.research.google.com/github/09334677600f/Code-for-Chapter-4/blob/main/%D9%85%D8%AF%D9%84%E2%80%8C%D9%87%D8%A7%DB%8C_%D8%AA%D8%AD%D9%84%DB%8C%D9%84_%D8%A8%D9%82%D8%A7_(%D8%A8%D8%AE%D8%B4_%D8%A7%D9%88%D9%84).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 kB 9.5 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=cf4c2e8804c71143c04c254ef1adc537af6657ccb2c7719418c1f9d7243145c4
  Stored in directory: /root/.cache/pip/wheels/50/37/21/0a719b9d89c635e89ff24bd93b862882ad675279552013b2fb
Successfully built autograd-gamma


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.optimize import minimize
import statsmodels.api as sm
from lifelines import WeibullAFTFitter, LogNormalAFTFitter, LogLogisticAFTFitter
from lifelines.utils import concordance_index
from lifelines.statistics import proportional_hazard_test

In [ ]:
df = pd.read_excel("/content/final_file1 (2) (1).xlsx")

In [ ]:

# 1. کتابخانه

import pandas as pd
import numpy as np
from lifelines import WeibullAFTFitter, LogNormalAFTFitter, LogLogisticAFTFitter
from scipy import stats
from sklearn.model_selection import train_test_split
from scipy.stats import chi2


# 2. پیش‌پردازش داده‌ها

columns_to_drop = ["ID", "Accident_ID", "تعداد تصادف",
                   "تاریخ", "زمان تماس", "ساعت رسیدن به محل حادثه"]

df = df.drop(columns=[c for c in columns_to_drop if c in df.columns])

X_cols = ["سن بیمار", "جنسیت", "زمان شب و روز",
          "نوع روز", "فصل", "ساعت اوج ترافیک"]
X = df[X_cols].copy()
y = df[["مدت زمان", "وضعیت"]].copy()

categorical_cols = ["جنسیت", "زمان شب و روز", "نوع روز", "فصل", "ساعت اوج ترافیک"]
X_cat = pd.get_dummies(X[categorical_cols], drop_first=True)
X_num = X[["سن بیمار"]]
X_processed = pd.concat([X_num, X_cat], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y["وضعیت"]
)

X_train = X_train.apply(pd.to_numeric, errors="coerce")
y_train["مدت زمان"] = pd.to_numeric(y_train["مدت زمان"], errors="coerce")
y_train["وضعیت"] = pd.to_numeric(y_train["وضعیت"], errors="coerce")
train_df = pd.concat([X_train, y_train], axis=1).dropna()
print("Train data shape:", train_df.shape)

# 6. Fit AFT models and compute LR test + Cox-Snell

aft_models = {
    "Log-Logistic": LogLogisticAFTFitter(),
    "Log-Normal": LogNormalAFTFitter(),
    "Weibull": WeibullAFTFitter()
}

simple_cols = ["سن بیمار", "مدت زمان", "وضعیت"]
train_df_simple = train_df[simple_cols]

for name, model in aft_models.items():
    print(f"\n===== {name} AFT =====")

    # Fit full model
    model.fit(train_df, duration_col="مدت زمان", event_col="وضعیت")
    model.print_summary()


    # Cox-Snell residuals

    cox_snell_residuals = []
    for i in range(len(train_df)):
        S_i = model.predict_survival_function(train_df.iloc[[i]][X_processed.columns],
                                              times=[train_df["مدت زمان"].iloc[i]])
        S_val = np.clip(S_i.values[0][0], 1e-10, 1)
        cox_snell_residuals.append(-np.log(S_val))
    cox_snell_residuals = np.array(cox_snell_residuals)

    ks_stat, p_value = stats.kstest(cox_snell_residuals, 'expon')
    print("Cox–Snell residuals KS test: statistic={:.4f}, p-value={:.6f}".format(ks_stat, p_value))


    # Likelihood Ratio Test (vs مدل ساده)

    simple_model = type(model)()  # همان نوع توزیع
    simple_model.fit(train_df_simple, duration_col="مدت زمان", event_col="وضعیت")

    ll_simple = simple_model.log_likelihood_
    ll_full = model.log_likelihood_
    LR_stat = -2 * (ll_simple - ll_full)
    df_chi2 = model.params_.shape[0] - simple_model.params_.shape[0]
    p_value_lr = chi2.sf(LR_stat, df_chi2)

    print(f"Likelihood Ratio Test vs simple model: LR={LR_stat:.4f}, df={df_chi2}, p-value={p_value_lr:.6f}")


Train data shape: (17682, 10)

===== Log-Logistic AFT =====


<lifelines.LogLogisticAFTFitter: fitted with 17682 total observations, 11597 right-censored observations>
             duration col = 'مدت زمان'
                event col = 'وضعیت'
   number of observations = 17682
number of events observed = 6085
           log-likelihood = -26058.61
         time fit was run = 2026-01-28 02:11:06 UTC

---
                                    coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param  covariate                                                                                                                    
alpha_ جنسیت_مونث                   0.02      1.03      0.02           -0.02            0.07                0.98                1.07
       زمان شب و روز_شب             0.05      1.05      0.02            0.01            0.09                1.01                1.10
       ساعت اوج ترافیک_خارج از اوج -0.20      0.82      0.03           -0.25           -0.15                0.78                0.86
       سن بیمار                    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
       فصل_تابستان                  0.53      1.70      0.04            0.45            0.61                1.57                1.85
       فصل_زمستان                  -0.34      0.72      0.04           -0.41           -0.26                0.66                0.77
       فصل_پاییز                    0.09      1.09      0.04            0.01            0.17                1.01                1.18
       نوع روز_اول هفته             0.03      1.04      0.02           -0.01            0.08                0.99                1.08
       Intercept                    2.93     18.82      0.05            2.84            3.03               17.05               20.77
beta_  Intercept                    0.43      1.53      0.01            0.41            0.45                1.50                1.57

                                    cmp to     z      p  -log2(p)
param  covariate                                                 
alpha_ جنسیت_مونث                     0.00  1.05   0.29      1.78
       زمان شب و روز_شب               0.00  2.48   0.01      6.23
       ساعت اوج ترافیک_خارج از اوج    0.00 -7.26 <0.005     41.26
       سن بیمار                       0.00 -1.12   0.26      1.92
       فصل_تابستان                    0.00 13.10 <0.005    127.77
       فصل_زمستان                     0.00 -8.34 <0.005     53.62
       فصل_پاییز                      0.00  2.20   0.03      5.17
       نوع روز_اول هفته               0.00  1.52   0.13      2.96
       Intercept                      0.00 58.38 <0.005       inf
beta_  Intercept                      0.00 38.64 <0.005       inf
---
Concordance = 0.62
AIC = 52137.21
log-likelihood ratio test = 1135.04 on 8 df
-log2(p) of ll-ratio test = 793.89

Cox–Snell residuals KS test: statistic=0.3986, p-value=0.000000
Likelihood Ratio Test vs simple model: LR=1131.8560, df=7, p-value=0.000000

===== Log-Normal AFT =====


<lifelines.LogNormalAFTFitter: fitted with 17682 total observations, 11597 right-censored observations>
             duration col = 'مدت زمان'
                event col = 'وضعیت'
   number of observations = 17682
number of events observed = 6085
           log-likelihood = -25695.76
         time fit was run = 2026-01-28 02:12:59 UTC

---
                                    coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param  covariate                                                                                                                    
mu_    جنسیت_مونث                   0.02      1.02      0.02           -0.02            0.06                0.98                1.07
       زمان شب و روز_شب             0.05      1.05      0.02            0.01            0.09                1.01                1.10
       ساعت اوج ترافیک_خارج از اوج -0.18      0.83      0.03           -0.23           -0.13                0.79                0.88
       سن بیمار                    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
       فصل_تابستان                  0.47      1.60      0.04            0.39            0.55                1.48                1.73
       فصل_زمستان                  -0.33      0.72      0.04           -0.41           -0.25                0.67                0.78
       فصل_پاییز                    0.08      1.08      0.04            0.00            0.16                1.00                1.17
       نوع روز_اول هفته             0.02      1.03      0.02           -0.02            0.07                0.98                1.07
       Intercept                    2.99     19.85      0.05            2.89            3.08               18.04               21.83
sigma_ Intercept                    0.09      1.09      0.01            0.06            0.11                1.07                1.11

                                    cmp to     z      p  -log2(p)
param  covariate                                                 
mu_    جنسیت_مونث                     0.00  0.90   0.37      1.44
       زمان شب و روز_شب               0.00  2.51   0.01      6.38
       ساعت اوج ترافیک_خارج از اوج    0.00 -6.99 <0.005     38.41
       سن بیمار                       0.00 -1.02   0.31      1.70
       فصل_تابستان                    0.00 11.94 <0.005    106.68
       فصل_زمستان                     0.00 -8.31 <0.005     53.20
       فصل_پاییز                      0.00  2.03   0.04      4.56
       نوع روز_اول هفته               0.00  1.15   0.25      2.00
       Intercept                      0.00 61.39 <0.005       inf
sigma_ Intercept                      0.00  8.19 <0.005     51.78
---
Concordance = 0.62
AIC = 51411.52
log-likelihood ratio test = 1049.38 on 8 df
-log2(p) of ll-ratio test = 732.44

Cox–Snell residuals KS test: statistic=0.4168, p-value=0.000000
Likelihood Ratio Test vs simple model: LR=1046.6072, df=7, p-value=0.000000

===== Weibull AFT =====


<lifelines.WeibullAFTFitter: fitted with 17682 total observations, 11597 right-censored observations>
             duration col = 'مدت زمان'
                event col = 'وضعیت'
   number of observations = 17682
number of events observed = 6085
           log-likelihood = -26663.08
         time fit was run = 2026-01-28 02:14:53 UTC

---
                                     coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param   covariate                                                                                                                    
lambda_ جنسیت_مونث                   0.03      1.04      0.03           -0.01            0.08                0.99                1.09
        زمان شب و روز_شب             0.03      1.03      0.02           -0.01            0.08                0.99                1.08
        ساعت اوج ترافیک_خارج از اوج -0.21      0.81      0.03           -0.27           -0.15                0.76                0.86
        سن بیمار                    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
        فصل_تابستان                  0.60      1.82      0.04            0.51            0.68                1.67                1.98
        فصل_زمستان                  -0.28      0.76      0.04           -0.36           -0.19                0.70                0.82
        فصل_پاییز                    0.15      1.16      0.04            0.06            0.23                1.06                1.26
        نوع روز_اول هفته             0.00      1.00      0.02           -0.04            0.05                0.96                1.05
        Intercept                    3.35     28.53      0.05            3.25            3.46               25.67               31.70
rho_    Intercept                    0.15      1.16      0.01            0.13            0.17                1.14                1.19

                                     cmp to     z      p  -log2(p)
param   covariate                                                 
lambda_ جنسیت_مونث                     0.00  1.38   0.17      2.59
        زمان شب و روز_شب               0.00  1.40   0.16      2.63
        ساعت اوج ترافیک_خارج از اوج    0.00 -7.11 <0.005     39.68
        سن بیمار                       0.00 -0.83   0.41      1.30
        فصل_تابستان                    0.00 13.44 <0.005    134.37
        فصل_زمستان                     0.00 -6.52 <0.005     33.73
        فصل_پاییز                      0.00  3.38 <0.005     10.42
        نوع روز_اول هفته               0.00  0.18   0.86      0.22
        Intercept                      0.00 62.33 <0.005       inf
rho_    Intercept                      0.00 14.19 <0.005    149.40
---
Concordance = 0.62
AIC = 53346.15
log-likelihood ratio test = 1014.40 on 8 df
-log2(p) of ll-ratio test = 707.35

Cox–Snell residuals KS test: statistic=0.4438, p-value=0.000000
Likelihood Ratio Test vs simple model: LR=1012.5232, df=7, p-value=0.000000


In [ ]:

# 1.کتابخانه
import pandas as pd
import numpy as np

from lifelines import LogLogisticAFTFitter
from scipy import stats
from sklearn.model_selection import train_test_split


# 1. حذف ستون‌های غیر ضروری

columns_to_drop = [
    "ID", "Accident_ID", "تعداد تصادف",
    "تاریخ", "زمان تماس", "ساعت رسیدن به محل حادثه"
]

df = df.drop(columns=[c for c in columns_to_drop if c in df.columns])

# 2. تعریف متغیر مستقل و وابسته
X_cols = [
    "سن بیمار", "جنسیت", "زمان شب و روز",
    "نوع روز", "فصل", "ساعت اوج ترافیک"
]

X = df[X_cols].copy()
y = df[["مدت زمان", "وضعیت"]].copy()

# 3. One-Hot
categorical_cols = [
    "جنسیت", "زمان شب و روز",
    "نوع روز", "فصل", "ساعت اوج ترافیک"
]

X_cat = pd.get_dummies(X[categorical_cols], drop_first=True)
X_num = X[["سن بیمار"]]

X_processed = pd.concat([X_num, X_cat], axis=1)


# 4. Train / Test split

X_train, X_test, y_train, y_test = train_test_split(
    X_processed,
    y,
    test_size=0.2,
    random_state=42,
    stratify=y["وضعیت"]
)

X_train = X_train.apply(pd.to_numeric, errors="coerce")
y_train["مدت زمان"] = pd.to_numeric(y_train["مدت زمان"], errors="coerce")
y_train["وضعیت"] = pd.to_numeric(y_train["وضعیت"], errors="coerce")

train_df = pd.concat([X_train, y_train], axis=1).dropna()

print("Train data shape:", train_df.shape)


# 5. Fit Log-Logistic AFT model

loglogistic = LogLogisticAFTFitter()

loglogistic.fit(
    train_df,
    duration_col="مدت زمان",
    event_col="وضعیت"
)

print("\n===== Log-Logistic AFT Summary =====")
loglogistic.print_summary()


# 6. Cox–Snell residuals

surv_fns = loglogistic.predict_survival_function(
    train_df[X_processed.columns],
    times=train_df["مدت زمان"]
)

#  S(t_i | x_i)
S_t = np.array([
    surv_fns.iloc[i, i] for i in range(len(train_df))
])


S_t = np.clip(S_t, 1e-10, 1)

cox_snell_residuals = -np.log(S_t)


# 7. Kolmogorov–Smirnov Test
# H0: Cox–Snell residuals ~ Exp(1)

ks_stat, p_value = stats.kstest(cox_snell_residuals, 'expon')

print("\n===== Goodness-of-Fit Test =====")
print("Kolmogorov–Smirnov test on Cox–Snell residuals")
print(f"KS statistic: {ks_stat:.4f}")
print(f"p-value: {p_value:.6f}")


Train data shape: (17682, 10)

===== Log-Logistic AFT Summary =====


<lifelines.LogLogisticAFTFitter: fitted with 17682 total observations, 11597 right-censored observations>
             duration col = 'مدت زمان'
                event col = 'وضعیت'
   number of observations = 17682
number of events observed = 6085
           log-likelihood = -26058.61
         time fit was run = 2026-01-28 02:05:26 UTC

---
                                    coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
param  covariate                                                                                                                    
alpha_ جنسیت_مونث                   0.02      1.03      0.02           -0.02            0.07                0.98                1.07
       زمان شب و روز_شب             0.05      1.05      0.02            0.01            0.09                1.01                1.10
       ساعت اوج ترافیک_خارج از اوج -0.20      0.82      0.03           -0.25           -0.15                0.78                0.86
       سن بیمار                    -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
       فصل_تابستان                  0.53      1.70      0.04            0.45            0.61                1.57                1.85
       فصل_زمستان                  -0.34      0.72      0.04           -0.41           -0.26                0.66                0.77
       فصل_پاییز                    0.09      1.09      0.04            0.01            0.17                1.01                1.18
       نوع روز_اول هفته             0.03      1.04      0.02           -0.01            0.08                0.99                1.08
       Intercept                    2.93     18.82      0.05            2.84            3.03               17.05               20.77
beta_  Intercept                    0.43      1.53      0.01            0.41            0.45                1.50                1.57

                                    cmp to     z      p  -log2(p)
param  covariate                                                 
alpha_ جنسیت_مونث                     0.00  1.05   0.29      1.78
       زمان شب و روز_شب               0.00  2.48   0.01      6.23
       ساعت اوج ترافیک_خارج از اوج    0.00 -7.26 <0.005     41.26
       سن بیمار                       0.00 -1.12   0.26      1.92
       فصل_تابستان                    0.00 13.10 <0.005    127.77
       فصل_زمستان                     0.00 -8.34 <0.005     53.62
       فصل_پاییز                      0.00  2.20   0.03      5.17
       نوع روز_اول هفته               0.00  1.52   0.13      2.96
       Intercept                      0.00 58.38 <0.005       inf
beta_  Intercept                      0.00 38.64 <0.005       inf
---
Concordance = 0.62
AIC = 52137.21
log-likelihood ratio test = 1135.04 on 8 df
-log2(p) of ll-ratio test = 793.89


===== Goodness-of-Fit Test =====
Kolmogorov–Smirnov test on Cox–Snell residuals
KS statistic: 0.3986
p-value: 0.000000


In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# پیش‌پردازش
columns_to_drop = ["ID", "Accident_ID", "تعداد تصادف", "تاریخ", "زمان تماس", "ساعت رسیدن به محل حادثه"]
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns], axis=1)

X_cols = ["سن بیمار", "جنسیت", "زمان شب و روز", "نوع روز", "فصل", "ساعت اوج ترافیک"]
X = df[X_cols].copy()
y = df[["مدت زمان", "وضعیت"]].copy()

categorical_cols = ["جنسیت", "زمان شب و روز", "نوع روز", "فصل", "ساعت اوج ترافیک"]
X_categorical = pd.get_dummies(X[categorical_cols], drop_first=True)

scaler = StandardScaler()
X_numeric = pd.DataFrame(scaler.fit_transform(X[["سن بیمار"]]), columns=["سن بیمار"], index=X.index)

X_processed = pd.concat([X_numeric, X_categorical], axis=1)

# تقسیم داده
X_train, X_test, y_train, y_test = train_test_split(
    X_processed, y, test_size=0.2, random_state=42, stratify=y["وضعیت"]
)

X_train = X_train.apply(pd.to_numeric, errors='coerce').astype(float)
y_train["مدت زمان"] = pd.to_numeric(y_train["مدت زمان"], errors='coerce')
y_train["وضعیت"] = pd.to_numeric(y_train["وضعیت"], errors='coerce')

# آماده‌سازی داده برای برازش
t = y_train["مدت زمان"].values
delta = 1 - y_train["وضعیت"].values  # فرض: 1 = مشاهده‌شده (event), 0 = censored

X = X_train.values
n, p = X.shape
X = np.hstack([np.ones((n,1)), X])  # اضافه کردن intercept

# تابع‌های log-likelihood برای هر توزیع
def weibull_loglik(params, t, delta, X):
    beta = params[:-2]
    sigma = np.exp(params[-2])  # مقیاس
    k = np.exp(params[-1])  # شکل
    mu = X @ beta
    z = (np.log(t) - mu) / sigma
    loglik = delta * (np.log(k) - np.log(sigma) + (k-1) * (np.log(t) - mu - np.log(sigma)) - np.exp(k * z)) + (1 - delta) * (-np.exp(k * z))
    return -np.sum(loglik)

def lognormal_loglik(params, t, delta, X):
    beta = params[:-1]
    sigma = np.exp(params[-1])
    mu = X @ beta
    z = (np.log(t) - mu) / sigma
    loglik = delta * (-np.log(sigma) - np.log(t) - 0.5 * z**2 - np.log(np.sqrt(2 * np.pi))) + (1 - delta) * np.log(1 - stats.norm.cdf(z))
    return -np.sum(loglik)

def loglogistic_loglik(params, t, delta, X):
    beta = params[:-1]
    sigma = np.exp(params[-1])
    mu = X @ beta
    z = (np.log(t) - mu) / sigma
    loglik = delta * (np.log(1/sigma) + np.log(1/t) + z - 2 * np.log(1 + np.exp(z))) + (1 - delta) * (-np.log(1 + np.exp(z)))
    return -np.sum(loglik)

dist_functions = {
    'Weibull': weibull_loglik,
    'LogNormal': lognormal_loglik,
    'LogLogistic': loglogistic_loglik
}

params_report = {}
lrt_results = {}
ks_results = {}

for name, loglik_fn in dist_functions.items():
    # برازش مدل کامل
    if name == 'Weibull':
        init_params = np.concatenate((np.log(np.mean(t)) * np.ones(p+1), [0.0, 0.0]))  # initial بهتر
    else:
        init_params = np.concatenate((np.log(np.mean(t)) * np.ones(p+1), [0.0]))

    res = minimize(loglik_fn, init_params, args=(t, delta, X), method='Nelder-Mead', tol=1e-6)
    params_full = res.x
    loglik_full = -res.fun

    # مدل خالی (بدون کوواریت)
    X_null = np.ones((n,1))
    if name == 'Weibull':
        init_null = [np.log(np.mean(t)), 0.0, 0.0]
    else:
        init_null = [np.log(np.mean(t)), 0.0]
    res_null = minimize(loglik_fn, init_null, args=(t, delta, X_null), method='Nelder-Mead', tol=1e-6)
    loglik_null = -res_null.fun

    # آزمون نسبت درستنمایی
    lrt_stat = 2 * (loglik_full - loglik_null)
    df_freedom = p
    lrt_p = stats.chi2.sf(lrt_stat, df_freedom)
    lrt_results[name] = {'آماره LRT': lrt_stat, 'p-value LRT': lrt_p}

    # برآورد پارامترها
    beta = params_full[:-2] if name == 'Weibull' else params_full[:-1]
    intercept = beta[0]
    coefs = beta[1:]
    sigma = np.exp(params_full[-2] if name == 'Weibull' else params_full[-1])
    shape = np.exp(params_full[-1]) if name == 'Weibull' else None
    params_report[name] = {'intercept': intercept, 'ضرایب کوواریت‌ها': coefs.tolist(), 'مقیاس (sigma)': sigma, 'شکل (shape)': shape}

    # محاسبه residuals Cox-Snell برای KS
    if name == 'Weibull':
        mu = X @ beta
        r = (t / np.exp(mu)) ** shape
    elif name == 'LogNormal':
        mu = X @ beta
        r = -stats.norm.logcdf((np.log(t) - mu) / sigma)
    elif name == 'LogLogistic':
        mu = X @ beta
        r = np.log(1 + (t / np.exp(mu)) ** (1/sigma))
    uncens_r = r[delta == 1]
    ks_stat, ks_p = stats.kstest(uncens_r, 'expon')
    ks_results[name] = {'آماره KS': ks_stat, 'p-value KS': ks_p}

# گزارش نتایج
for name in dist_functions:
    print(f"\n--- مدل {name} ---")
    print("برآورد پارامترها:")
    print(params_report[name])
    print("\nآزمون نسبت درستنمایی:")
    print(lrt_results[name])
    print("\nآزمون Kolmogorov-Smirnov:")
    print(ks_results[name])

/tmp/ipython-input-3638514667.py:49: RuntimeWarning: divide by zero encountered in log
  loglik = delta * (np.log(k) - np.log(sigma) + (k-1) * (np.log(t) - mu - np.log(sigma)) - np.exp(k * z)) + (1 - delta) * (-np.exp(k * z))
/tmp/ipython-input-3638514667.py:49: RuntimeWarning: invalid value encountered in multiply
  loglik = delta * (np.log(k) - np.log(sigma) + (k-1) * (np.log(t) - mu - np.log(sigma)) - np.exp(k * z)) + (1 - delta) * (-np.exp(k * z))
/tmp/ipython-input-3638514667.py:49: RuntimeWarning: overflow encountered in exp
  loglik = delta * (np.log(k) - np.log(sigma) + (k-1) * (np.log(t) - mu - np.log(sigma)) - np.exp(k * z)) + (1 - delta) * (-np.exp(k * z))
/tmp/ipython-input-3638514667.py:48: RuntimeWarning: overflow encountered in divide
  z = (np.log(t) - mu) / sigma
/tmp/ipython-input-3638514667.py:48: RuntimeWarning: divide by zero encountered in divide
  z = (np.log(t) - mu) / sigma
/tmp/ipython-input-3638514667.py:49: RuntimeWarning: invalid value encountered in add
  


--- مدل Weibull ---
برآورد پارامترها:
{'intercept': np.float64(1433917.3239731733), 'ضرایب کوواریت\u200cها': [-90818.36241734831, -41150.35357790245, 332903.6926423579, -596190.8202395476, -248320.68549106654, -319319.9135716099, 195113.3815897426, -656240.1471844062], 'مقیاس (sigma)': np.float64(4.104601037597891e-303), 'شکل (shape)': np.float64(1.3126784e-316)}

آزمون نسبت درستنمایی:
{'آماره LRT': np.float64(-39990942961.6093), 'p-value LRT': np.float64(1.0)}

آزمون Kolmogorov-Smirnov:
{'آماره KS': np.float64(0.8353884625334138), 'p-value KS': np.float64(0.0)}

--- مدل LogNormal ---
برآورد پارامترها:
{'intercept': np.float64(2.6513793391327676), 'ضرایب کوواریت\u200cها': [0.006083077377738333, 0.006583265043540162, -0.010214537965613155, -0.04338144907472735, 0.022123879986168563, 0.14120185991149464, 0.08350932717386672, 0.0391894648241594], 'مقیاس (sigma)': np.float64(1.0047211648824133), 'شکل (shape)': None}

آزمون نسبت درستنمایی:
{'آماره LRT': np.float64(-15486.850346401043), 'p-

In [ ]:


import pandas as pd

# 1. جدول برآورد پارامترها
param_rows = []
for name in params_report:
    p = params_report[name]
    intercept = round(p['intercept'], 4)
    sigma     = round(p['مقیاس (sigma)'], 4)
    shape     = round(p['شکل (shape)'], 4) if p['شکل (shape)'] is not None else "-"


    coefs_str = ", ".join([f"{c:.4f}" for c in p['ضرایب کوواریت‌ها'][:5]])
    if len(p['ضرایب کوواریت‌ها']) > 5:
        coefs_str += f", ... ({len(p['ضرایب کوواریت‌ها'])} متغیر)"

    param_rows.append({
        'توزیع': name,
        'Intercept': intercept,
        'مقیاس (σ)': sigma,
        'شکل': shape,
        'ضرایب کوواریت‌ها (نمونه)': coefs_str
    })

param_df = pd.DataFrame(param_rows)
print("\n" + "═"*80)
print("جدول ۱ – برآورد پارامترهای مدل‌های AFT")
print("═"*80)
print(param_df.to_string(index=False))

# 2. جدول آزمون نسبت درستنمایی (LRT)
lrt_rows = []
for name in lrt_results:
    r = lrt_results[name]
    lrt_rows.append({
        'توزیع': name,
        'آماره χ²': round(r['آماره LRT'], 4),
        'درجه آزادی': p,
        'p-value': f"{r['p-value LRT']:.2e}"
    })

lrt_df = pd.DataFrame(lrt_rows)
print("\n" + "═"*80)
print("جدول ۲ – آزمون نسبت درستنمایی (Likelihood Ratio Test)")
print("═"*80)
print("H₀: همه ضرایب کوواریت‌ها صفر هستند")
print(lrt_df.to_string(index=False))

# 3. جدول آزمون Kolmogorov-Smirnov روی residuals
ks_rows = []
for name in ks_results:
    r = ks_results[name]
    ks_rows.append({
        'توزیع': name,
        'آماره KS': round(r['آماره KS'], 4),
        'p-value': f"{r['p-value KS']:.2e}"
    })

ks_df = pd.DataFrame(ks_rows)
print("\n" + "═"*80)
print("جدول ۳ – آزمون Kolmogorov-Smirnov روی residuals Cox-Snell")
print("═"*80)
print("H₀: توزیع فیت‌شده با داده‌ها سازگار است")
print(ks_df.to_string(index=False))

# 4. خلاصه تفسیری کوتاه
print("\n" + "─"*80)
print("خلاصه و پیشنهاد")
print("─"*80)

best_by_aic = min(results, key=lambda k: results[k]['AIC'])
print(f"• بهترین مدل بر اساس AIC: {best_by_aic}")
print(f"• همه مدل‌ها: p-value LRT بسیار کوچک → کوواریت‌ها اثر معنی‌داری دارند.")
print(f"• آزمون KS: در داده‌های بزرگ و ناهمگن معمولاً p-value کوچک می‌شود.")
print("  → حتی اگر p-value KS کوچک باشد، مدل LogNormal یا LogLogistic اغلب بهترین تعادل را دارد.")
print("  → پیشنهاد: مدل با کمترین AIC + بررسی C-index یا پیش‌بینی روی داده تست")


════════════════════════════════════════════════════════════════════════════════
جدول ۱ – برآورد پارامترهای مدل‌های AFT
════════════════════════════════════════════════════════════════════════════════
      توزیع    Intercept  مقیاس (σ)  شکل                                                         ضرایب کوواریت‌ها (نمونه)
    Weibull 1433917.3240     0.0000  0.0 -90818.3624, -41150.3536, 332903.6926, -596190.8202, -248320.6855, ... (8 متغیر)
  LogNormal       2.6514     1.0047    -                          0.0061, 0.0066, -0.0102, -0.0434, 0.0221, ... (8 متغیر)
LogLogistic       2.5798     1.0399    -                           0.0032, 0.0084, 0.0082, -0.0242, 0.1253, ... (8 متغیر)

════════════════════════════════════════════════════════════════════════════════
جدول ۲ – آزمون نسبت درستنمایی (Likelihood Ratio Test)
════════════════════════════════════════════════════════════════════════════════
H₀: همه ضرایب کوواریت‌ها صفر هستند
      توزیع      آماره χ²                                 